![MmSCel Logo](https://storage.googleapis.com/kaggle-competitions/kaggle/38128/logos/header.png?t=2022-08-08-22-48-50)


<div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:250%;">
    <p style="padding: 4px;text-align: center;color:white;"><b>Complete EDA of MmSCel Integration Data</b></p>
    <p style="padding: 4px;text-align:center; color:white;"><b>Table of contents</b></p>
</div> 

1. [Intro](#1)
2. [What data is collected](#2)
    1. [Data from Multiome Test](#21)
        1. [Chromatin accessibility data](#211)
        2. [Gene Expression Data](#212)
    2. [Data from CITEseq test](#22)
        1. [Gene Expression Data](#221)
        2. [Surface Protein Level Data](#222)
3. [How is data collected](#3)
4. [Data centric view](#4)
    1. [Chromatin Accessibility](#41)
    2. [Gene Expression](#42)
        1. [From Multiome](#421)
        2. [From CITEseq](#422)
    3. [Surface Protein Levels](#43)
5. [How to Submit to competition](#5)

In [1]:
# installs
!pip install --quiet tables

# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gc
import tables as tb
from tqdm.auto import tqdm
import re
import seaborn as sns
sns.set_theme()
import random

# set paths
DATA_DIR = os.environ['DATA_DIR']
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

/home/len/miniconda3/envs/eda_env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1 Intro <a id="1"></a>

The goal of this competition is to better understand the relationship between different modalities in cells. The goal of this notebook is to gain a better understanding of the associated data. This equips us with the knowledge needed to make good decisions about model design and data layout.

**This is a work in progress. If any aspect needs clarification, please let me know. My understanding of genetics is very limited. Feel free to point out anything that is false.**

During transcription in cells, there is a known **flow of information**. DNA must be accessible to produce RNA. Produced RNA is used as a template to build proteins. Therefore, one could assume that we can use knowledge about the accessibility of DNA to predict future states of RNA and that we can use knowledge about RNA to predict the concentration of proteins in the future. In this challenge, we want to learn more about this relationship between DNA, RNA, and proteins. We thus need to capture information about three distinct properties of a cell:
* chromatin accessibility (DNA)
* gene expression (RNA)
* surface protein levels

Before we have a look at how the information about those properties of a cell is laid out, we must note that the methods used to obtain the data do not capture all properties at once. **We have two distinct methods for testing**. The first one is the "10x Chromium Single Cell Multiome ATAC + Gene Expression" short "multiome" test. The second one is the "10x Genomics Single Cell Gene Expression with Feature Barcoding technology" short "citeseq" test.

With the multiome test, we can measure **chromatin accessibility and gene expression**. With the citeseq test, we can measure **gene expression and surface protein levels**. Therefore, we will have data about chromatin accessibility and surface protein levels once (from multiome and citeseq, respectively). And we will have data about gene expression two times, once from each test.

To get a better understanding of the data we will look at it from 3 perspectives. The first thing we will look at is, what data is actually collected. Questions we will adress here are: How many measurments do we get from each cell? How to interpret the measurements we have? How accurate is the data collected? This can be thought of as a biological view of the problem and is the main focus of **chapter two**. We will not go into details here how the general data landscape looks like.

The **third chapter** will look at how the data is collected. The focus of this chapter will be the metadata file. Questions of concern will be: On what days was the data collected? What different Splits do we obtain? How much data is in each split?

After having an understanding of the collection methods and results of the collection process we will have a look at the outcome of the collection. Namely the different datasets or splits, we are presented with. We will look at the properties of the splits as a whole and compute well known statistics about them. All of this can be found in **chapter four**.

This notebook will be concluded with the **final chapter** on necessary details to compete in the competition. We will have a look at what data needs to be submitted and in what format it has to be submitted.


# 2 What Data is collected <a id="2"></a>

Let's zoom in at what data is actually collected. As we already mentioned in the introduction we have two different test methods that each measure two modalities of the introspected cell. Therefore we have four measurments in total. Each of those four measurments receives its own subchapter. We will start witht the measurments from the multiome test and than look at the data from the citeseq test.

<a id="21"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:200%;">
    <p style="padding: 4px;color:white;"><b>2.A Data from Multiome test</b></p>
</div> 

The first two measurments we are looking at are obtained by the multiome test. Multiome test is short for **Chromium Single Cell Multiome ATAC + Gene Expression**. As the name of the test suggest we get measurments for ATAC (assay for transpoase-accessible chromatin) and for Gene Expression Data. The tests are done by the Company 10x Genomics. The description of the test can be found [here](https://www.10xgenomics.com/products/single-cell-multiome-atac-plus-gene-expression). The test has a single cell resolution. That means every data point we obtain can be mapped to a single cell.

<a id="211"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>2.A.a Chromatin accessibility data</b></p>
</div>

First we will have a look at the Chromatin Accessibility data, since that's also the first data in the information flow of a cell. Here we will just be concerened with a single data point and look at what data is measured. Below you see the ATAC data obtained by multiome for a single cell.

In [2]:
df_multi_train_x = pd.read_hdf(FP_MULTIOME_TRAIN_INPUTS,start=0,stop=1)
df_multi_train_x

FileNotFoundError: File /home/len/Data/train_multi_inputs.h5 does not exist

As we can see, each individual cell is identified by a cell_id in this case "56390cf1b95e". We then have 228942 measurments for each cell that are named something like "STUFF:NUMBER-NUMBER". STUFF is actually the name of a chromosome. Let's have a look at what kind of chromosomes we have:



In [ ]:
print(sorted(list({i[:i.find(':')] for i in df_multi_train_x.columns})))

We actually find the chromosomes we expect, namely chr1-chr22, the 22 chromosomes humans have (called autosomes), and also chrX and chrY, being the gender-specific chromosomes. What about the ones starting with KI and GL? According to a quick internet search, those are unplaced genes. They most likely are part of the human genome, but we don't know yet on which chromosome they are. 

What about the numbers after the chromosome name? These numbers identify a certain region on that chromosome. These regions are called peaks and multiple peaks can contribute to the same gene. For a nice and short introduction check out this [5 min Youtube Video on ATAC sequencing.](https://www.youtube.com/watch?v=uuxpyhGNDsk)

I want to look into three things in further detail:
* Why do we obtain numbers as floats? From what I understand so far, I would expect the data to be binary. Whether a peak is accessible or not.
* Can peak regions actually overlap? From the method explained in the video, I would assume not, but this needs to be checked.
* If multiple peaks contribute to a single gene, the assumption is obvious that if one peak is accessible, neighboring peaks are likely accessible as well. Is this the case?

<a id="212"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>2.A.b Gene Expression Data</b></p>
</div>

The mere presence of a genome in a cell does not alter that cell or the organism that it is part of. The information in the genome must be "interpreted" in order for any change to take place. In other words, a gene's information is employed to create a functioning gene product. Either a protein or non-coding RNA might be this. The most fundamental method of converting genomic information into gene products is called gene expression. The information about the genes that are active in a particular cell or the regions of the genome that are involved in the function of the cell may be inferred from the gene expression data we are utilizing for this challenge. An example of that data for the cell 56390cf1b95e is provided below.

In [ ]:
df_multi_train_y = pd.read_hdf(FP_MULTIOME_TRAIN_TARGETS, start=0, stop=1)

print(f"Different starts of names: {sorted(list({i[:10] for i in df_multi_train_y.columns}))}")
print(f"Different lengths of names: {len(df_multi_train_y.columns.str.len().unique())}")

df_multi_train_y

As we can see, each ID begins with ENSG and is then followed by 5 zeroes. It is called Ensambl ID. The general form is ENS(species)(object type)(identifier).(version). ENS tells us that we are looking at an Ensembl ID. By convention, the species field for human genes is left blank. The object type for genes is G. The identifier appears to always be 11 decimal places long. Additionally, it appears that our data lacks any version specifications. 

I suggest viewing [this video](https://www.youtube.com/watch?v=bKIpDtJdK8Q) for a thorough explanation of the entire transcription and translation process we are exploring for this challenge. It provides a thorough summary of the role mRNA plays in the procedure. According to what I understand, each gene on the genome corresponds to an mRNA molecule, and the numbers in our dataset indicate how much mRNA for each corresponding gene is present in the sampled cell. More mRNA is indicated by higher values.

<a id="22"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:200%;">
    <p style="padding: 4px;color:white;"><b>2.B Data from CITEseq test</b></p>
</div>

By now we have quite a good understanding of what data can be obtained by the multiome test. Now we want to have a look at the **Single Cell Gene Expression with Feature Barcoding technology**, or short **CITEseq** test. The test data we are provided with is again from the company 10x Genomics. Information about the test can be found on the company's website [here](https://support.10xgenomics.com/permalink/getting-started-single-cell-gene-expression-with-feature-barcoding-technology). It is a test to "reveal cell surface protein and gene expression from the same cell". Since we are already familiar with gene expression data, let's first look at that.

<a id="221"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>2.B.a Gene Expression Data</b></p>
</div>

We are again loading a single sample from the test set to get an idea about the features we are presented with:

In [ ]:
df_cite_train_x = pd.read_hdf(FP_CITE_TRAIN_INPUTS,start=0,stop=1)
df_cite_train_x

The first thing we notice is that the start of the gene_id looks much like what we have seen in the multiome data, but there is a new suffix. So what is it about the suffix?

Checking the Ensembl ID of gene_id on [ensembl.org](https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000121410;r=19:58345178-58353492) (in this case for ENSG00000121410) we see that the suffix is actually the name of the gene. As we will see in the next code cell, the gene_id is unique even without this suffix, so it looks like redundant information for now.

In [ ]:
gene_ids_citeseq = set([i[:i.find("_")] for i in df_cite_train_x.columns])
len(gene_ids_citeseq)

Stripping of the suffixes still produces 22050 unique ids.

Let's check for overlap in both datasets about gene expression:

In [ ]:
gene_ids_multiome = set(df_multi_train_y.columns)

print(f"Elements in Set Union: {len(gene_ids_citeseq | gene_ids_multiome)}")
print(f"Elements in Set Intersection: {len(gene_ids_citeseq & gene_ids_multiome)}")
print(f"multiome has {len(gene_ids_multiome - gene_ids_citeseq)} unique gene ids.")
print(f"Citeseq has {len(gene_ids_citeseq - gene_ids_multiome)} unique gene ids.")

We have quite a huge overlap. More than 18k genes are found in both datasets. But still there are quite a few genes that are unique in each dataset.

<a id="222"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>2.B.b Surface protein level data</b></p>
</div>

Lastly, we will have a look at the surface protein levels data gathered by citeseq.

In [ ]:
df_cite_train_y = pd.read_hdf(FP_CITE_TRAIN_TARGETS, start=0, stop=1)
df_cite_train_y

Compared to what we have seen so far, the number of columns in this data set is quite small. We have measurements of 140 features per cell. Most of the names start with CD, which is short for "Cluster of differentiation". CDs are used to classify surface molecules a cell expresses. This information can then be used to get an idea of what kind of cell is present, or what function this cell is supposed to serve in the body (I am not sure if my understanding here is even remotely accurate).

Here we see all the measured proteins that do not start with CD:

In [ ]:
surface_portein_ids = set(df_cite_train_y.columns)
regex = re.compile(r'^CD.*')
{p for p in surface_portein_ids if not regex.match(p)}

Iterestingly some of the names start with Rat or Mouse. All of our donors are humans. We will later see if we actually find non zero values for these proteins in the test set.

Now having a rough understanding what data is collected by each test and how to interpret the collected data we will have a look at how data is collected. This will be the main focus of chapter 3.

# 3 How is data collected <a id="3"></a>

On top of the actual data we have information about how the data is collected. This metadata is stored in the file metadata.csv. The file contains one row for each cell in the dataset and provides some additional information about that cell.

In [ ]:
df_meta = pd.read_csv(FP_CELL_METADATA).set_index("cell_id")
df_meta

For each cell, we see the **day** column tells us on which day the test was performed. The test that was actually performed is shown in the **technology** column. Note that experiments started on day 1, therefore the first tests were performed one day after the cells were injected with Neupogen for the first time ([compare here](https://allcells.com/research-grade-tissue-products/mobilized-leukopak/)). For each of the four donors, we have a **donor** ID giving us information about the origin of the cell. Lastly, there is a **cell_type** column. The cell types are labels assigned by humans. They might be imprecise and this information is not available for test data since it would be possible to draw conclusions about surface protein levels, for example. It is not clear if we can make use of that later (maybe we can use it for creating balanced splits, but we will see).

[jirkaborovec](https://www.kaggle.com/jirkaborovec) already composed a great analysis of the metadata in this [notebook](https://www.kaggle.com/code/jirkaborovec/mmscel-inst-eda-stat-predictions). I will copy some parts here for readability and add some comments myself.

In [ ]:
fig, axarr = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))
for i, col in enumerate(["donor", "day", "technology"]):
    _= df_meta[[col]].value_counts().plot.pie(ax=axarr[i], autopct='%1.1f%%', ylabel=col)

As we can see, the cell data is pretty balanced. We have almost an equal number of cells from each donor (the big numbers in the first picture are the donor ids). Also, the days of the experiment were fairly balanced. The last day, day 10, only receives an 11% share of the cells. Day 10 is also the only day not present in the train data at all! Also, the train set does not contain any data from donor 27678!

We have slightly more data available for the multinome test. It is not the worst since our model also has to predict many more features for that test.

The distribution of data in general is pretty well balanced (e. g., the number of tests taken / test / day is well distributed). For a more in-depth analysis of the metadata, I highly recommend the already mentioned [notebook](https://www.kaggle.com/code/jirkaborovec/mmscel-inst-eda-stat-predictions).

In [ ]:
sns.catplot(
    data=df,
    x="donor",
    hue="technology",
    col="day",
    kind="count",
)

# 4 Data centric view <a id="4"></a>

Having a good understanding about what properties of a cell we measure and how data is collected lets now look at the properties of the datasets we are provided with. The following three subchapters will focus on that. One Subchapter for each property.

<a id="41"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:200%;">
    <p style="padding: 4px;color:white;"><b>4.A Chromatin Accessibility</b></p>
</div>

This part of the analysis is actually quite challenging. The problem we face is due to layout of memory. We receive the memory arranged by rows. This means, for a single cell all the features about that cell are arranged together in memory. But in some cases we are interested in properties of features and not properties of cells. To give an example, it would be really interesting to know the mean and std deviation of every feature present in our dataset. Since we cannot load the whole dataset into memory at once, we have to load batches. The problem is, that when loading say 10 feature columns, we have to look at (cell count) different locations in storage. This takes considerable time, since we have about 100k cells. If we wanted to load the features of 10 cells into RAM we would get results almost immediately. 

Therefore I created a dataset with the data transposed (its uploaded on kaggle as mmscel-data-transposed). And also two notebooks that calculate features of rows and columns. They are named "cell-feature-generation" and "feature-feature-generation". Both are public as well.

Let's start with looking at "cell features". This is the easier part of the analysis.

In [ ]:
# lets load the calculated features into a dataframe
df_train = pd.read_csv('../input/cell-feature-generation/train_multi_inputs_cell_features.csv').rename(columns={'Unnamed: 0': 'cell_id'})
df_train.insert(1, 'set', 'train')
df_test = pd.read_csv('../input/cell-feature-generation/test_multi_inputs_cell_features.csv').rename(columns={'Unnamed: 0': 'cell_id'})
df_test.insert(1, 'set', 'test')

df = pd.concat([df_train, df_test]).reset_index(drop=True)
del df_train, df_test; gc.collect()

df

In [ ]:
print(f"Each cell has at least {df['count_non_zero'].min()} genes with non-zero accessibility values and a maximum of {df['count_non_zero'].max()}.")
print(f"On average there are {round(df['count_non_zero'].mean())} genes with non-zero accessibility values in each cell.")
print(f"The mean non-zero value for each cell ranges between {df['mean_non_zero'].min()} and {df['mean_non_zero'].max()}.\nThe average mean non-zero over all cells is {df['mean_non_zero'].mean()}.")

In [ ]:
sns.displot(df, x="count_non_zero", binwidth=500, hue='set')
# hier evtl kind=kde irgendwo verwenden (weil die hoehen sonst unterschiedlich sind wegen der verschiedenen set groessen)

In [ ]:
sns.relplot(
    data=df,
    x='count_non_zero', y='2_tups', col='set', s=1
)
# Eine Idee waere hier die erwartete Anzahl von zwei Tupeln/ drei tupeln und so weiter gegen die anzahl von tatsaechlich vorliegenden n tupeln zu plotten

Next we will see something really interesting. As mentioned before the locations on the genome we are looking at are called peeks. Multiple peeks can constitute to a single gene. Therefore one can assume, that we find groups of neighbooring peeks, that are all non-zero (the logic behind that is, that reading DNA cares about reading a whole gene and not about reading human defined peek regions). So lets check for that. We will take 100 random samples from the test set.

In [ ]:
for i in tqdm(range(100)):
    s_num = random.randint(0,100000)
    df = pd.read_hdf(FP_MULTIOME_TRAIN_INPUTS,start=s_num,stop=s_num+1)
    val_c_df = (df.iloc[0].le(0).astype(int).cumsum().value_counts() - 1)
    snake_count = val_c_df[val_c_df != 0].value_counts()
    if i == 0:
        res_df = snake_count
    else:
        res_df = res_df.add(snake_count, fill_value=0)

res_df

This is definitely not what I expected. Most of the accessible parts of the genome do not have any accessible neighbors. As I am not a geneticist I don't know why this is the case. Maybe someone else can explain.

After having a look at "features of cells" we will have a look at "features of features". 
It will take almost the same time to load 10 Feature Columns or 1000 Feature columns. Therefore we will set our batch size as high as possible.

In [ ]:
# this block of code is long and ugly to read. It basically just loads data columnwise and
# calculates some features of the columns. After that those features are stored in a dataframe
# for later analysis
with tb.open_file(FP_MULTIOME_TRAIN_INPUTS, 'r') as fileh:
    # get handles for the individual datablocks in hdf5 file
    values = fileh.root.train_multi_inputs.block0_values
    column_names = fileh.root.train_multi_inputs.block0_items
    cell_ids = fileh.root.train_multi_inputs.axis1
    
    # create lists of all cell_ids and genome_peeks
    cell_ids_list = [i.decode('UTF-8') for i in cell_ids[:]]
    genome_peeks_list = [i.decode('UTF-8') for i in column_names[:]]
    
    # the features we want to calculate for every genome_peek
    target_features = [
        'count_non_zero',
        'max_value',
        'min_value',
        'sum_values',
        'mean_non_zero',
        'std_dev_non_zero',
    ]
    
    # list to hold batch DFs
    res_df_list = []
    
    batchsize = 5_000
    iterations = int(np.ceil(len(genome_peeks_list) / batchsize))
    
    # TODO change 2 to iterations
    for i in tqdm(range(2)):
        # this is the computational expensive part. Even if we just want to look at
        # 10k columns, we have to look at 100k different storage locations
        df = pd.DataFrame(values[:,i*batchsize:(i+1)*batchsize].T,
                          index=genome_peeks_list[i*batchsize: (i+1)*batchsize],
                          columns=cell_ids_list)
        
        tmp_df = pd.DataFrame(index=genome_peeks_list[i*batchsize: (i+1)*batchsize], columns=target_features)
        
        # this is not so expensive. Once the memory is loaded we can compute the features we want
        tmp_df['count_non_zero'] = df.gt(0).sum(axis=1).astype(pd.Int64Dtype())
        tmp_df['max_value'] = df.max(axis=1).astype(pd.Float32Dtype())
        tmp_df['min_value'] = df.min(axis=1).astype(pd.Float32Dtype())
        tmp_df['sum_values'] = df.sum(axis=1).astype(pd.Float64Dtype())
        tmp_df['mean_non_zero'] = df[df.gt(0)].mean(axis=1).astype(pd.Float32Dtype())
        tmp_df['std_dev_non_zero'] = df[df.gt(0)].std(axis=1).astype(pd.Float32Dtype())
        res_df_list.append(tmp_df)
    
peeks_feature_df = pd.concat(res_df_list)
del res_df_list
del tmp_df
gc.collect()

In [ ]:
# check for peeks that are zero for every single cell
peeks_feature_df[peeks_feature_df['count_non_zero'] == 0]

As we can see, there are 10 peek region, that are zero for every single cell. We can remove those features from our dataset.

In [ ]:
print(f"Values range from {peeks_feature_df['min_value'].min()} to {peeks_feature_df['max_value'].max()}")


In [ ]:
sns.jointplot(
    data=peeks_feature_df,
    x="mean_non_zero", y="max_value",
    kind="kde"
)

In [ ]:
sns.kdeplot(data=peeks_feature_df[['mean_non_zero', 'max_value']].values)

<a id="42"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:200%;">
    <p style="padding: 4px;color:white;"><b>4.B Gene Expression</b></p>
</div>

<a id="221"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>4.B.a From Multiome</b></p>
</div>

<a id="221"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:150%;">
    <p style="padding: 4px;color:white;"><b>4.B.b From CITEseq</b></p>
</div>

<a id="43"></a><div style="color:white;display:fill;
            background-color:#3bb2d6;font-size:200%;">
    <p style="padding: 4px;color:white;"><b>4.C Surface Protein Levels</b></p>
</div>

# 5 How to submit to competition <a id="5"></a>

# X Notes

There are still some open questions in the text we need to address. 

* Correct Information about peaks in chromatin accessibility data
* include memory information for each data frame
* review ordering (single cell view, (metadata), data science centric view, kaggle specific data)
* Add Data overview (size of Datasets)